# Working with Unstructured Grid Data

UXarray offers support for loading and representing unstructured grids
by providing Xarray-like functionality paired with new routines that
are specifically written for operating on unstructured grids.


## Grid Definition and Data Variables

When working with Unstructured Grids, the grid definition and data variables
are often store as separate files. This means that there are multiple
separate files that need to be read and linked together to represent the
entire dataset.

For example, the following sample dataset is taken from NOAA Geoflow project,
which is made up of 4 files (1 grid definition and 3 data files).
Special thanks to John Clyne, Shilpi Gupta, and the VAPOR team for providing these data!

```
geoflow-small
│   grid.nc
│   v1.nc
│   v2.nc
│   v3.nc
```

## Grid Conventions

Given the complexity of Unstructured Grids, there are many different ways of representing the underlying topology and structure. These representations are referred to as conventions, and they outline
the required connectivity variables, naming conventions, data types, naming conventions, and many other specifications.

UXarray uses the UGRID conventions internally to represent Unstructured Grids.



## Opening The Grid and Data Files with UXarray
Uxarray provides function to read in grid definition and data variable netCCDF files together.

First, data paths and UXarray import

In [ ]:
import uxarray as ux

In [ ]:
# Base data path
base_path = "../../test/meshfiles/ugrid/geoflow-small/"

# Path to Grid file
grid_path = base_path + "grid.nc"

# Paths to Data Variable files
var_names = ['v1.nc', 'v2.nc', 'v3.nc']

data_paths = [base_path + name for name in var_names]

In [ ]:
uxds_single = ux.open_dataset(grid_path, data_paths[0])
uxds_single

In [ ]:
uxds_complete = ux.open_mfdataset(grid_path, data_paths)
uxds_complete

In [ ]:
uxds_complete.uxgrid

In [ ]:
uxds_single.uxgrid == uxds_complete.uxgrid

### Reading only the Unstructured Grid Topology (i.e. without specific data) with UXarray

A single grid definition file can be loaded with UXarray to explore the topology:

In [ ]:
# Construct a UXarray Grid object from `grid_ds`
grid = ux.open_grid(grid_path)

In [ ]:
grid

In [ ]:
grid._ds

### Opening data files with the Unstructured Grid Topology they correspond to with UXarray

Uxarray can open data along with the grid topology via Xarray-like dataset opening functions!

In [ ]:
uxds_v1 = ux.open_dataset(grid_path, data_paths[0])

In [ ]:
uxds_v1

In [ ]:
uxds_v1.uxgrid._ds

As can be seen, the Grid object has its own `grid.ds` of type `xarray.Dataset`
to define the grid topology. However, the Grid object has further attributes,
variables, and functions to specify the unstructured grid and be executed on
it, which can be explored in the next notebooks.